In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ibge.silver

In [0]:
# Import das funções
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType

In [0]:
#Lendo DataFrame na Silver
instrucao_df = spark.table("ibge.bronze.instrucao")

In [0]:
# Remover header residual (linha descritiva)
instrucao_linha_df = (
    instrucao_df
    .filter(col("NC").isNotNull())
    .filter(col("NC") != "Nível Territorial (Código)")
)

# Selecionar e transformar
instrucao_final_df = (
    instrucao_linha_df
    .select(
        col("NC").cast(IntegerType()).alias("cod_nivel_territorial"),
        col("D1C").cast(IntegerType()).alias("cod_municipio"),
        col("D1N").alias("municipio"),
        col("D3N").cast(IntegerType()).alias("ano"),
        col("D4N").alias("sexo"),
        col("D5N").alias("cor_raca"),
        col("D6N").alias("nivel_instrucao"),
        col("D7N").alias("grupo_idade"),
        col("V").cast(DoubleType()).alias("valor")
    )
)

# Salvar na Silver
(
    instrucao_final_df.write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable("ibge.silver.instrucao")
)